<a href="https://colab.research.google.com/github/Indahfitrialita/LLM-Open-Course-by-Kalbe-Indah/blob/main/Fill_Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [ ]:
!git config --global user.email "indahfitrialita8@gmail.com"
!git config --global user.name "Indah Fitrialita"

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> DistilBERT number of parameters: {round(distilbert_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> DistilBERT number of parameters: 67M'
'>>> BERT number of parameters: 110M'


In [ ]:
text = "This is a great [MASK]."

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> This is a great deal.'
'>>> This is a great success.'
'>>> This is a great adventure.'
'>>> This is a great idea.'
'>>> This is a great feat.'


# Dataset

Dataset yang digunakan merupakan dataset Large Movie Review Dataset, yang merupakan kumpulan data ulasan film yang sering digunakan sebagai referensi untuk menguji model analisis sentimen.

In [ ]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
sample = imdb_dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['text']}'")
    print(f"'>>> Label: {row['label']}'")


'>>> Review: There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...'
'>>> Label: 1'

'>>> Review: This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stu

# Pre-Processing

Semua contoh dalam dataset diconcatenate menjadi satu kesatuan, dan kemudian korpus lengkap dibagi menjadi potongan-potongan (chunks) yang memiliki ukuran yang sama.

In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [ ]:
tokenizer.model_max_length

512

Set ukuran chunk menjadi 128 agar muat dalam memori GPU

In [ ]:
chunk_size = 128

In [ ]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 363'
'>>> Review 1 length: 304'
'>>> Review 2 length: 133'


Lalu concatenate semua contoh


In [ ]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 800'


Selanjutnya akan displit concatenate reviews ke chunks=128. Hasilnya adalah kamus chunk untuk tiap feature:

In [ ]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 32'


Karena panjang chunk yang terakhir lebih kecil dari sebelum-sebelumnya, maka ada dua cara yang bisa dilakukan, yaitu:


*   Hapus chunk terakhir jika lebih kecil dari chunk_size=128
*   Tambahkan padding pada potongan terakhir sehingga panjangnya sama dengan chunk_size.



Kita pilih yang pertama

In [ ]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

Pada fungsi 'group_texts()', kolom baru dengan nama 'labels' dibuat. Kolom tersebut merupakan salinan dari kolom 'input_ids'. Tujuannya adalah untuk memprediksi token-token yang acak dimask pada batch input, dan dengan membuat kolom label, kita menyediakan ground truth agar model dapat belajar.

Selanjutnya terapkan group_texts() pada dataset-token menggunakan fungsi 'Dataset.map():

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

Terlihat bahwa setelah mengconcatenate dan menjadikan dalam bentuk chunk telah menghasilkan banyak contoh lebih dari 25000 untuk data train dan test.

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

"as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. while my countrymen mind find it shocking, in reality sex and nudity are a major staple in swedish cinema. even ingmar bergman,"

Terlihat dua ulasan film yang saling tumpang tindih.

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["labels"])

"as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. while my countrymen mind find it shocking, in reality sex and nudity are a major staple in swedish cinema. even ingmar bergman,"

# Fine-tuning DistrilBERT dengan Trainer API


Dalam fine-tuning model bahasa yang telah dimask, diperlukan collator data khusus yang dapat secara acak melakukan masking pada beberapa token dalam setiap batch teks.

Gunakan package DataCollatorForLanguageModeling. Lalu tokenizer dan mlm_probability argument yang menentukan seberapa besar pecahan token yang akan dimask.  Kita akan memilih 15%, yang merupakan jumlah yang digunakan untuk BERT dan pilihan umum dalam literatur:

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Melihat bagaimana random masking bekerja:
Karena data collator mengharapkan daftar kamus, di mana setiap kamus mewakili satu potongan teks berkesinambungan, pertama lakukan dulu iterasi pada dataset sebelum memberikan batch ke collator. Hapus kunci "word_ids" untuk collator data:


In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] i rented i am curious - yellow [MASK] my video store [MASK] of [MASK] [MASK] controversy that surrounded it [MASK] it [MASK] first released in 1967. i also heard that [MASK] first [MASK] was seized by [MASK] [MASK] s. customs if it ever [MASK] to enter this country, therefore [MASK] [MASK] [MASK] of films considered " controversial " i really had to [MASK] this for myself. < br / > < suspiciously / > the [MASK] is [MASK] around [MASK] young swedish drama [MASK] named lena who [MASK] to [MASK] everything she can about life. in particular she thriller to focus her attentions to making some sort of documentary on what [MASK] average swede thought about certain political issues such'

'>>> as [MASK] vietnam war and race issues in thesdale states. in between asking politicians and ordinary denizens [MASK] [MASK] about their opinions on politics, [MASK] has [MASK] [MASK] her drama teacher, [MASK], and [MASK] men. < br / > < br / > what kills me about i am curious - [MASK] [MASK] 

Terlihat [MASK] token secara random terletak pada sembarang lokasi pada teks tersebut. Hal itu akan menjadi token yang model prediksi selama training.

Saat melakukan training untuk masked language modeling, teknik yang dapat digunakan adalah *whole word masking* yang digunakan untuk melakukan masking pada seluruh kata bersamaan, bukan hanya token-tokennya.

Step adalam melakukan whole word masking adalah:
Buat data collator sendiri,
Data collator hanya fungsi yang mengambil list sample dan mengconvert list dan sample tersebut ke dalam batch, sehingga perlu membuat map antara indeks kata dan token yang sesuai, lalu secara acak memutuskan kata-kata mana yang akan dimask.

In [ ]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

Coba pada sample yang sama seperti sebelumnya:

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] i rented i am curious - [MASK] [MASK] my video store because of all the controversy that surrounded it when [MASK] was [MASK] released in 1967 [MASK] i also [MASK] that [MASK] first it [MASK] seized [MASK] [MASK]. s. customs if it ever tried [MASK] enter this country, therefore being a [MASK] [MASK] films [MASK] [MASK] controversial " i really had [MASK] see [MASK] for myself [MASK] [MASK] br / > [MASK] br [MASK] > the plot [MASK] [MASK] [MASK] a [MASK] swedish drama student [MASK] lena who [MASK] [MASK] [MASK] everything she can about [MASK]. in particular she wants to focus her attentions to [MASK] some sort of documentary on what the average [MASK] [MASK] thought [MASK] certain political issues such'

'>>> [MASK] [MASK] vietnam war and race issues in [MASK] united states. in [MASK] asking politicians [MASK] ordinary denizens [MASK] stockholm [MASK] their [MASK] on [MASK] [MASK] she has sex with her drama teacher, [MASK], and married men. < [MASK] [MASK] [MASK] [MASK] br 

Sekarang bahwa kita memiliki dua data collator, langkah-langkah fine-tuning yang tersisa adalah standar.

Karena train dapat memakan waktu cukup lama, maka akan dipotong ukuran set train menjadi beberapa ribu contoh dengan menggunakan fungsi Dataset.train_test_split()

In [ ]:
train_size = 10_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

Terlihat bahwa ukuran set train diatur menjadi 10000 contoh dan set validasi sebesar 10%

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Setelah masuk ke HuggingFace, kita dapat menetapkan nilai-nilai argumen yang diperlukan untuk Trainer agar sesuai dengan kebutuhan pelatihan model tertentu yang sedang kita lakukan.

In [ ]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-imdb",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

'trainer.train()' sudah siap dijalankan.

# Perplexity for Language Model

Perplexity merupakan metric evaluasi yang umum digunakan untuk mengevaluasi kinerja model bahasa.


*   Untuk mengukur kualitas model bahasa adalah dengan menghitung probabilitas yang diberikannya kepada kata berikutnya dalam semua kalimat dari set test.
*  Hitung perplexity dari model yang telah di-pretrain dengan menggunakan fungsi Trainer.evaluate() untuk menghitung cross-entropy loss pada set pengujian dan kemudian mengambil eksponensial dari hasilnya:




In [ ]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 21.94


Nilai perplexity yang lebih rendah artinya model bahasa akan lebih baik. Untuk menurunkan nilainya dapat dilakukan dengan fine-tuning!


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.702400,2.496466
2,2.579200,2.428034
3,2.535400,2.450834


TrainOutput(global_step=471, training_loss=2.604898126768205, metrics={'train_runtime': 172.9682, 'train_samples_per_second': 173.442, 'train_steps_per_second': 2.723, 'total_flos': 994208670720000.0, 'train_loss': 2.604898126768205, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 11.15


In [ ]:
trainer.push_to_hub()

events.out.tfevents.1706490596.f94a3a075f4a.461.0:   0%|          | 0.00/6.00k [00:00<?, ?B/s]

events.out.tfevents.1706490804.f94a3a075f4a.461.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Indah1/distilbert-base-uncased-finetuned-imdb/commit/8204412dc27f2f8beffbbb5707c82b0772e47827', commit_message='End of training', commit_description='', oid='8204412dc27f2f8beffbbb5707c82b0772e47827', pr_url=None, pr_revision=None, pr_num=None)

# Fine-tuning DistilBERT with Accelerate

DataCollatorForLanguageModeling juga dapat digunakan sebagai random masking untuk tiap evaluasi. Untuk mengeliminasi randomness tersebut adalah dengan membuat masking hanya sekali pada seluruh data test dan menggunakan data collator untuk mengumpulkan batch selama evaluasi.

In [ ]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

Selanjutnya terapkan fungsi 'insert_random_mask' tersebut pada test set dan drop kolom yang tidak dimask sehingga dapat menggantinya dengan yang sudah dimask.

In [ ]:
downsampled_dataset = downsampled_dataset.remove_columns(["word_ids"])
eval_dataset = downsampled_dataset["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=downsampled_dataset["test"].column_names,
)
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Gunakan default_data_collator untuk set evaluasi:

In [ ]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

batch_size = 64
train_dataloader = DataLoader(
    downsampled_dataset["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_data_collator
)

Accelerate:


1.   Load versi baru dari model yang sudah di pretrain
2.   Siapkan objek-objek yang digunakan pada Accelerator
3.   Konfigurasi dataloaders
4.   Tentukan epoch



1. Load versi terbaru dari model yang telah dipretrain

In [ ]:
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

2. Tentukan Optimizer

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

3. Siapkan objek-objek yang akan digunakan untuk training

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

4. Tentukan epoch

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

Sebelum training, buat dulu repositori model di Huggin Face

In [ ]:
from huggingface_hub import get_full_repo_name

model_name = "distilbert-base-uncased-finetuned-imdb"
repo_name = get_full_repo_name(model_name)
repo_name

'Indah1/distilbert-base-uncased-finetuned-imdb'

Selanjutnya buat dan clone repo menggunakan Repository class:

In [ ]:
from huggingface_hub import Repository

output_dir = repo_name
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Indah1/distilbert-base-uncased-finetuned-imdb into local empty directory.


Download file model.safetensors:   0%|          | 8.00k/256M [00:00<?, ?B/s]

Download file runs/Jan29_01-09-35_f94a3a075f4a/events.out.tfevents.1706490596.f94a3a075f4a.461.0: 100%|#######…

Download file runs/Jan29_01-09-35_f94a3a075f4a/events.out.tfevents.1706490804.f94a3a075f4a.461.1: 100%|#######…

Download file training_args.bin: 100%|##########| 4.49k/4.49k [00:00<?, ?B/s]

Clean file runs/Jan29_01-09-35_f94a3a075f4a/events.out.tfevents.1706490596.f94a3a075f4a.461.0:  17%|#7        …

Clean file runs/Jan29_01-09-35_f94a3a075f4a/events.out.tfevents.1706490804.f94a3a075f4a.461.1: 100%|##########…

Clean file training_args.bin:  22%|##2       | 1.00k/4.49k [00:00<?, ?B/s]

Clean file model.safetensors:   0%|          | 1.00k/256M [00:00<?, ?B/s]

Lalu load full training dan evaluation:

In [ ]:
from tqdm.auto import tqdm
import torch
import math

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/471 [00:00<?, ?it/s]

>>> Epoch 0: Perplexity: 11.259870233779559
>>> Epoch 1: Perplexity: 11.048357354029154
>>> Epoch 2: Perplexity: 11.048357354029154


Terlihat bahwa training model menghasilkan hasil yang konsisten karena makin rendah skor Perplexity, menunjukkan kinerja model yang lebih baik.




# Using Our Fine-tune Model

gunakan pipeline fill-mask untuk download modelnya di hugging face

In [ ]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="huggingface-course/distilbert-base-uncased-finetuned-imdb"
)

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> this is a great film.
>>> this is a great movie.
>>> this is a great idea.
>>> this is a great deal.
>>> this is a great adventure.


model dengan jelas telah menyesuaikan bobotnya untuk memprediksi kata-kata yang lebih kuat terkait dengan film!